In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Nov  6 14:57:08 PST 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables

`%env` are best for bash

In [9]:
# Set workding directory
%env wd=/home/sam/analyses/20191106_swoose_pgen_feature_stats
wd="/home/sam/analyses/20191106_swoose_pgen_feature_stats"

# File download commands
%env rsync_gannet=gannet:/volume2/web/Atumefaciens/20191105_swoose_pgen_v074_renaming/
%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate  https://gannet.fish.washington.edu/Atumefaciens/20191105_swoose_pgen_v074_renaming/

# Input/output files
%env exon_gff=Panopea-generosa-v1.0.a4.exon.gff3
%env exon_bed=Panopea-generosa-v1.0.a4.exon.bed
%env gene_gff=Panopea-generosa-v1.0.a4.gene.gff3
%env gene_bed=Panopea-generosa-v1.0.a4.gene.bed
%env introns_bed=Panopea-generosa-v1.0.a4.introns.bed
%env intergenic_bed=Panopea-generosa-v1.0.a4.intergenic.bed

# Programs
%env intersectbed=/home/sam/programs/bedtools-2.28.0/bin/intersectBed

# Set list of column header names
bed_header = ['scaffold', 'start', 'end']

env: wd=/home/sam/analyses/20191106_swoose_pgen_feature_stats
env: rsync_gannet=gannet:/volume2/web/Atumefaciens/20191105_swoose_pgen_v074_renaming/
env: wget_command=--directory-prefix=$/home/sam/analyses/20191106_swoose_pgen_feature_stats --quiety --no-directories --no-check-certificate  https://gannet.fish.washington.edu/Atumefaciens/20191105_swoose_pgen_v074_renaming/
env: exon_gff=Panopea-generosa-v1.0.a4.exon.gff3
env: exon_bed=Panopea-generosa-v1.0.a4.exon.bed
env: gene_gff=Panopea-generosa-v1.0.a4.gene.gff3
env: gene_bed=Panopea-generosa-v1.0.a4.gene.bed
env: introns_bed=Panopea-generosa-v1.0.a4.introns.bed
env: intergenic_bed=Panopea-generosa-v1.0.a4.intergenic.bed
env: intersectbed=/home/sam/programs/bedtools-2.28.0/bin/intersectBed


### Import Python modules

In [10]:
import fnmatch
import os
import pandas

In [11]:
%%bash
mkdir --parents ${wd}

In [12]:
cd {wd}

/home/sam/analyses/20191106_swoose_pgen_feature_stats


### Download files

In [18]:
%%bash
# Create array of files from list
files_array=(${exon_gff} ${gene_gff} ${intergenic_bed} ${introns_bed})

for file in "${files_array[@]}"
do
  rsync \
  --archive \
  --progress \
  --verbose \
  "${rsync_gannet}${file}" \
  .
done

ls -lh

# If need to download via wget, uncomment lines in the cell below
# for file in "${files_array[@]}"
# do 
#  "${wget_command}${file}"
# done
# ls -lh ${wd}

receiving incremental file list

sent 11 bytes  received 79 bytes  180.00 bytes/sec
total size is 57,148,140  speedup is 634,979.33
receiving incremental file list
Panopea-generosa-v1.0.a4.gene.gff3
      9,888,459 100%  110.95MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 9,889,785 bytes  19,779,630.00 bytes/sec
total size is 9,888,459  speedup is 1.00
receiving incremental file list

sent 11 bytes  received 83 bytes  62.67 bytes/sec
total size is 1,019,566  speedup is 10,846.45
receiving incremental file list
Panopea-generosa-v1.0.a4.introns.bed
      4,579,012 100%  111.97MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 4,579,691 bytes  9,159,442.00 bytes/sec
total size is 4,579,012  speedup is 1.00
total 76M
-rw-rw-r-- 1 sam sam   6.7M Nov  6 15:21 Panopea-generosa-v1.0.a4.exon.bed
-rw-rw-r-- 1 sam users  55M Nov  5 08:55 Panopea-generosa-v1.0.a4.exon.gff3
-rw-rw-r-- 1 sam users 9.5M Nov  5 08:55 Panopea-generosa-v1.0.a4.gene.gff3
-rw-rw-r-- 1 sam users 99

### Create gene and exon BED files

Too lazy to write for loop

In [19]:
%%bash
echo "Preview GFF: ${exon_gff}"
echo ""
head "${exon_gff}"
echo ""
echo "---------------------"
echo ""

# Create BED file
# Skip first three lines (header) of GFF
# Print scaffold name, start, and stop positions; tab-delimited
awk 'NR > 3 {print $1"\t"$4"\t"$5}' "${exon_gff}" \
> "${exon_bed}"

echo "Preview BED: ${exon_bed}"
echo ""
head "${exon_bed}"
echo ""
echo "---------------------"
echo ""

echo "Preview GFF: ${gene_gff}"
echo ""
head "${gene_gff}"
echo ""
echo "---------------------"
echo ""

# Create BED file
# Skip first three lines (header) of GFF
# Print scaffold name, start, and stop positions; tab-delimited
awk 'NR > 3 {print $1"\t"$4"\t"$5}' "${gene_gff}" \
> "${gene_bed}"

echo "Preview BED: ${gene_bed}"
echo ""
head "${gene_bed}"
echo ""

Preview GFF: Panopea-generosa-v1.0.a4.exon.gff3

##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
Scaffold_01	GenSAS_5d9637f372b5d-publish	exon	2	125	.	+	.	ID=PGEN_.00g000010.m01.exon01;Name=PGEN_.00g000010.m01.exon01;Parent=PGEN_.00g000010.m01;original_ID=21510-PGEN_.00g234140.m01.exon1;Alias=21510-PGEN_.00g234140.m01.exon1
Scaffold_01	GenSAS_5d9637f372b5d-publish	exon	1995	2095	.	+	.	ID=PGEN_.00g000010.m01.exon02;Name=PGEN_.00g000010.m01.exon02;Parent=PGEN_.00g000010.m01;original_ID=21510-PGEN_.00g234140.m01.exon2;Alias=21510-PGEN_.00g234140.m01.exon2
Scaffold_01	GenSAS_5d9637f372b5d-publish	exon	3325	3495	.	+	.	ID=PGEN_.00g000010.m01.exon03;Name=PGEN_.00g000010.m01.exon03;Parent=PGEN_.00g000010.m01;original_ID=21510-PGEN_.00g234140.m01.exon3;Alias=21510-PGEN_.00g234140.m01.exon3
Scaffold_01	GenSAS_5d9637f372b5d-publish	exon	4651	4719	.	+	.	ID=PGEN_.00g000010.m01.exon04;Name=PGEN_.00g000010.m01.exon04;Parent=PGEN_.00g000

### Determine percent of genes containing introns

In [20]:
%%bash
# Count the number of genes
num_genes=$(cat ${gene_bed} | wc -l)
echo "${num_genes} found in ${gene_bed}"
echo ""
echo "----------------------------"
echo ""

# Determine number of genes with no intron overlap (that's the -v option below)
num_genes_no_introns=$(${intersectbed} -v -a ${gene_bed} -b ${introns_bed} | wc -l)
echo "${num_genes_no_introns} genes with no introns"
echo ""
echo "----------------------------"
echo ""

# Determine percentage of genes with introns.
# Sets "scale" to get desired number of decimal places.
percent_genes_w_introns=$(echo "scale=4;  1 - (${num_genes_no_introns} / ${num_genes})" | bc)


echo "${percent_genes_w_introns} of genes contain introns"


34947 found in Panopea-generosa-v1.0.a4.gene.bed

----------------------------

9772 genes with no introns

----------------------------

.7204 of genes contain introns


### Stats summaries for the BED files

In [21]:
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*.bed'):
        print('\n' * 2)
        print(file)
        print("-------------------------")
        
        # Import GFF.
        # Skip first 3 rows (gff header lines) and indicate file is tab-separated
        bed=pandas.read_csv(file, header=None, sep="\t")
        
        # Rename columns
        bed.columns = bed_header
        
        # Subtract start value from end value.
        # Have to add 1 so that sequence length can't equal zero (i.e. adjust for 1-based counting system)
        bed['seqlength'] = bed.apply(lambda position: position['end'] - position['start'] + 1, axis=1)
        
        # Apply functions in list to seqlength column
        bed_stats = bed['seqlength'].agg(['mean', 'min', 'median', 'max', 'sum'])
        
        # Print table of calculation type and the result
        for calc_type, calcs in bed_stats.iteritems():
            print(calc_type, calcs, sep='\t')




Panopea-generosa-v1.0.a4.intergenic.bed
-------------------------
mean	16393.607065170105
min	2.0
median	8114.0
max	370672.0
sum	565235178.0



Panopea-generosa-v1.0.a4.gene.bed
-------------------------
mean	10811.04461041005
min	166.0
median	4464.0
max	283066.0
sum	377813576.0



Panopea-generosa-v1.0.a4.introns.bed
-------------------------
mean	2184.529027548067
min	2.0
median	1199.0
max	93104.0
sum	338130141.0



Panopea-generosa-v1.0.a4.exon.bed
-------------------------
mean	201.4769876772451
min	3.0
median	133.0
max	13221.0
sum	47741987.0


In [22]:
### Clean up

In [24]:
%%bash
# Cleanup
rm ${exon_bed} ${exon_gff} ${gene_bed} ${gene_gff} ${intergenic_bed} ${introns_bed}

ls -l

total 0


rm: cannot remove 'Panopea-generosa-v1.0.a4.exon.gff3': No such file or directory
rm: cannot remove 'Panopea-generosa-v1.0.a4.gene.gff3': No such file or directory
rm: cannot remove 'Panopea-generosa-v1.0.a4.intergenic.bed': No such file or directory
rm: cannot remove 'Panopea-generosa-v1.0.a4.introns.bed': No such file or directory
